In [6]:
import numpy as np
import tensorflow as tf
from PIL import Image
from ModelPredict import compair
from tensorflow.keras.utils import image_dataset_from_directory

In [7]:
def Dataset_from_directory(data_dir, predict_dir, split):
	"""
	    
	Parameters
	----------
	train : str. Which dataset desired: AB,AC,BC
	test : str. corrisponding testdata: A,B,C 
	
	split : float. Fraction of validation split (0.2 =20%)
	Returns
	-------
	train_ds : tensorflow dataset for training. 
	pred_ds : tensorflow dataset for testing/predicting.
	"""

	train_ds = image_dataset_from_directory(
	data_dir,
	seed=53,
	image_size=(900, 900))

	val_ds = image_dataset_from_directory(
		data_dir,
	validation_split=split,
	subset="validation",
	seed=53,
	image_size=(900, 900))

	pred_ds= image_dataset_from_directory(
		predict_dir,
		seed = 53,
		image_size=(900, 900))

	return train_ds, val_ds, pred_ds

In [11]:
import tensorflow as tf
from PIL import Image

group = 'A'
data_dir = './Datasets/3_split'
predict_dir = './Datasets/cropped_jpgs/'


train_ds, val_ds, pred_ds = Dataset_from_directory(data_dir, predict_dir, 0.5)

normalization_layer = tf.keras.layers.Rescaling(1./255) # normalizing pixel intensity, 8 bit camera
pred_ds = pred_ds.map(lambda x, y: (normalization_layer(x), y)) # normalizing pixel intensity, 8 bit camera


model = tf.keras.models.load_model('classifier 3_split_final')
# model.summary()
# predict = model.predict(pred_ds)
# print(predict)

# test_accuracy = tf.keras.metrics.Accuracy()
# print(test_accuracy)
# test_loss = estimator.model.evaluate(train_ds)
# print("test set mse is %.2f" % test_loss)
# print('load complete')
# predictions = model.predict(pred_ds)

# matrix = metrics.confusion_matrix(y_test.argmax(axis=1), y_pred.argmax(axis=1))
# predictions = model(pred_ds, training=False)

RMSprop = tf.keras.optimizers.RMSprop(
    learning_rate=0.0001907, momentum=0.00082)

model.compile(
  optimizer=RMSprop,
  loss=tf.keras.losses.MeanSquaredError())

test_accuracy = tf.keras.metrics.Accuracy()
ds_test_batch = pred_ds

for (x, y) in ds_test_batch:
  # training=False is needed only if there are layers with different
  # behavior during training versus inference (e.g. Dropout).
  logits = model(x, training=True)
  prediction = tf.argmax(logits, axis=1, output_type=tf.int64)
  test_accuracy(prediction, y)

print("Test set accuracy: {:.3%}".format(test_accuracy.result()))

Found 12 files belonging to 3 classes.
Found 12 files belonging to 3 classes.
Using 6 files for validation.
Found 7 files belonging to 1 classes.
Test set accuracy: 100.000%


TypeError: predict() missing 1 required positional argument: 'x'